In [1]:
import sys
sys.path.append("../lib")
import os
from influxdb import InfluxDBClient, DataFrameClient
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
from pyutils import *
from pdutils import *
from pdpltutils import *



In [2]:
# Hardcode cloudlet IP and port for DB
CLOUDLET_IP = '128.2.208.248'
EPC_IP = '192.168.25.4'
UE_IP = '192.168.25.2'
CLOUDLET_PORT = 8086

IPMAP = {CLOUDLET_IP:"CLOUDLET",EPC_IP:"EPC",UE_IP:"UE"}

# ICMP databases
CLOUDLET_ICMP_DB = 'cloudleticmp'
WATERSPOUT_ICMP_DB = 'waterspouticmp'
UE_ICMP_DB = 'ueicmp'
SEG_DB = 'segmentation'
SEG_DB = 'segmentation'

### Functions

In [3]:
def dumpdf(df, rows=5):
    print(df.shape)
    print(df.head(rows))
    print(df.dtypes)

### Clients

In [4]:
''' Node clients '''
cloudlet_icmp_client = InfluxDBClient(host=CLOUDLET_IP, port=CLOUDLET_PORT, database=CLOUDLET_ICMP_DB)
df_cloudlet_icmp_client = DataFrameClient(host=CLOUDLET_IP, port=CLOUDLET_PORT, database=CLOUDLET_ICMP_DB)

waterspout_icmp_client = InfluxDBClient(host=CLOUDLET_IP, port=CLOUDLET_PORT, database=WATERSPOUT_ICMP_DB)
df_waterspout_icmp_client = DataFrameClient(host=CLOUDLET_IP, port=CLOUDLET_PORT, database=WATERSPOUT_ICMP_DB)

ue_icmp_client = InfluxDBClient(host=CLOUDLET_IP, port=CLOUDLET_PORT, database=UE_ICMP_DB)
df_ue_icmp_client = DataFrameClient(host=CLOUDLET_IP, port=CLOUDLET_PORT, database=UE_ICMP_DB)

measure = 'latency'

In [5]:
''' Segmentation Clients '''
seg_client = InfluxDBClient(host=CLOUDLET_IP, port=CLOUDLET_PORT, database=SEG_DB)
df_seg_client = DataFrameClient(host=CLOUDLET_IP, port=CLOUDLET_PORT, database=SEG_DB)

measure = 'uplink'
seg_ul_df = df_seg_client.query("select * from {}".format(measure))[measure]
measure = 'downlink'
seg_dl_df = df_seg_client.query("select * from {}".format(measure))[measure]
seqlst = list(set(seg_ul_df.sequence).intersection(seg_dl_df.sequence))
seg_df = pd.concat([seg_ul_df,seg_dl_df])
seg_df = seg_df[seg_df.sequence.isin(seqlst)].sort_values(['sequence','direction'],ascending=[True,False]).drop_duplicates()

dumpdf(seg_df)

(166, 11)
                                  cloudlet_proc direction  epc_cloudlet  \
2022-03-31 17:25:12.067579+00:00       0.000000    uplink      0.000290   
2022-03-31 17:25:12.067579+00:00       0.000045  downlink      0.000142   
2022-03-31 17:27:46.105357+00:00       0.000000    uplink      0.000311   
2022-03-31 17:27:46.105357+00:00       0.000044  downlink      0.000156   
2022-03-31 17:28:48.951001+00:00       0.000000    uplink      0.000138   

                                    offset       rtt    sequence  \
2022-03-31 17:25:12.067579+00:00  0.019276  0.024229  5416/10261   
2022-03-31 17:25:12.067579+00:00  0.019276  0.024229  5416/10261   
2022-03-31 17:27:46.105357+00:00  0.019276  0.026487  5568/49173   
2022-03-31 17:27:46.105357+00:00  0.019276  0.026487  5568/49173   
2022-03-31 17:28:48.951001+00:00  0.019276  0.041114  5630/65045   

                                         start   stepsum   ue_xran  \
2022-03-31 17:25:12.067579+00:00  1.648748e+09  0.024229  0.

### Queries

In [6]:
''' Node queries '''
measure = 'latency'
cloudlet_icmp_df = df_cloudlet_icmp_client.query("select * from {}".format(measure))[measure]
waterspout_icmp_df = df_waterspout_icmp_client.query("select * from {}".format(measure))[measure]
ue_icmp_df = df_ue_icmp_client.query("select * from {}".format(measure))[measure]

In [7]:
''' Segmentation queries '''
measure = 'uplink'
seg_ul_df = df_seg_client.query("select * from {}".format(measure))[measure]
measure = 'downlink'
seg_dl_df = df_seg_client.query("select * from {}".format(measure))[measure]
seqlst = list(set(seg_ul_df.sequence).intersection(seg_dl_df.sequence))
seg_df = pd.concat([seg_ul_df,seg_dl_df])
seg_df = seg_df[seg_df.sequence.isin(seqlst)].sort_values(['sequence','direction'],ascending=[True,False]).drop_duplicates()


In [8]:
display_all(seg_df)

(166, 11)


,cloudlet_proc,direction,epc_cloudlet,offset,rtt,sequence,start,stepsum,ue_xran,ue_xran_adjust,xran_epc
2022-03-31 17:25:12.067579+00:00,0.000000,uplink,0.000290,0.019276,0.024229,5416/10261,1.648748e+09,0.024229,0.025667,0.006391,0.000216
2022-03-31 17:25:12.067579+00:00,0.000045,downlink,0.000142,0.019276,0.024229,5416/10261,1.648748e+09,0.024229,-0.002286,0.016990,0.000155
2022-03-31 17:27:46.105357+00:00,0.000000,uplink,0.000311,0.019276,0.026487,5568/49173,1.648748e+09,0.026487,0.027894,0.008618,0.000221
2022-03-31 17:27:46.105357+00:00,0.000044,downlink,0.000156,0.019276,0.026487,5568/49173,1.648748e+09,0.026487,-0.002308,0.016968,0.000169
2022-03-31 17:28:48.951001+00:00,0.000000,uplink,0.000138,0.019276,0.041114,5630/65045,1.648748e+09,0.041114,0.042249,0.022973,0.000285
2022-03-31 17:28:48.951001+00:00,0.000045,downlink,0.000152,0.019276,0.041114,5630/65045,1.648748e+09,0.041114,-0.001973,0.017303,0.000218
2022-03-31 17:29:14.330027+00:00,0.000000,uplink,0.000344,0.019276,0.031823,5655/5910,1.648748e+09,0.031823,0.033229,0.013953,0.000193
2022-03-31 17:29:14.330027+00:00,0.000044,downlink,0.000154,0.019276,0.031823,5655/5910,1.648748e+09,0.031823,-0.002306,0.016970,0.000165
2022-03-31 17:29:42.729803+00:00,0.000000,uplink,0.000433,0.019276,0.042002,5683/13078,1.648748e+09,0.042002,0.043459,0.024183,0.000242
2022-03-31 17:29:42.729803+00:00,0.000035,downlink,0.000147,0.019276,0.042002,5683/13078,1.648748e+09,0.042002,-0.002516,0.016760,0.000202


In [9]:
tdfx = seg_df.copy()
tdfx['TIMESTAMP'] = tdfx.start.map(lambda epoch: pd.to_datetime(epoch, unit = 's'))
dumpdf(tdfx)

(166, 12)
                                  cloudlet_proc direction  epc_cloudlet  \
2022-03-31 17:25:12.067579+00:00       0.000000    uplink      0.000290   
2022-03-31 17:25:12.067579+00:00       0.000045  downlink      0.000142   
2022-03-31 17:27:46.105357+00:00       0.000000    uplink      0.000311   
2022-03-31 17:27:46.105357+00:00       0.000044  downlink      0.000156   
2022-03-31 17:28:48.951001+00:00       0.000000    uplink      0.000138   

                                    offset       rtt    sequence  \
2022-03-31 17:25:12.067579+00:00  0.019276  0.024229  5416/10261   
2022-03-31 17:25:12.067579+00:00  0.019276  0.024229  5416/10261   
2022-03-31 17:27:46.105357+00:00  0.019276  0.026487  5568/49173   
2022-03-31 17:27:46.105357+00:00  0.019276  0.026487  5568/49173   
2022-03-31 17:28:48.951001+00:00  0.019276  0.041114  5630/65045   

                                         start   stepsum   ue_xran  \
2022-03-31 17:25:12.067579+00:00  1.648748e+09  0.024229  0.

### Inspection

In [10]:
''' Get list of sequences in all three dataframes '''
seqminset = list(set(ue_icmp_df.sequence) \
    .intersection((set(cloudlet_icmp_df.sequence) \
    .intersection(set(waterspout_icmp_df.sequence)))))

print(seqminset)

['5815/46870', '5889/279', '5797/42262', '5843/54038', '5827/49942', '5818/47638', '5875/62230', '5817/47382', '5883/64278', '5821/48406', '5848/55318', '5819/47894', '5854/56854', '5824/49174', '5811/45846', '5857/57622', '5795/41750', '5849/55574', '5805/44310', '5655/5910', '5699/17174', '5856/57366', '5869/60694', '5851/56086', '5803/43798', '5822/48662', '5885/64790', '5833/51478', '5802/43542', '5830/50710', '5798/42518', '5873/61718', '5799/42774', '5858/57878', '5876/62486', '5816/47126', '5814/46614', '5416/10261', '5812/46102', '5868/60438', '5880/63510', '5823/48918', '5874/61974', '5846/54806', '5704/18454', '5859/58134', '5825/49430', '5829/50454', '5844/54294', '5683/13078', '5871/61206', '5800/43030', '5860/58390', '5853/56598', '5888/23', '5808/45078', '5804/44054', '5840/53270', '5831/50966', '5845/54550', '5809/45334', '5855/57110', '5796/42006', '5842/53782', '5807/44822', '5828/50198', '5872/61462', '5630/65045', '5841/53526', '5870/60950', '5801/43286', '5861/58646

In [11]:
''' Waterspout fix '''
tdfx = waterspout_icmp_df.copy()
tdfx['SRCNAME'] = tdfx.src.map(lambda col: IPMAP[col])
tdfx['DSTNAME'] = tdfx.dst.map(lambda col: IPMAP[col])
# dumpdf(tdfx[tdfx.sequence.isin(seqminset)],rows=7)
dumpdf(tdfx[tdfx.sequence.isin(seqminset)][['sequence','src','dst','SRCNAME','DSTNAME']],rows=7)

thresh = 0.1
tdfy = tdfx[(tdfx.SRCNAME == "UE") & (tdfx.DSTNAME == 'EPC')]

tdfy['DELTA'] = tdfy.epoch - tdfy.epoch.shift(1)
tdfy['DELTA'] = tdfy.DELTA.map(lambda col: col if col < thresh else np.NAN)
tdfy = tdfy.dropna(subset=['DELTA'])
dumpdf(tdfy)
print(tdfy.DELTA.median())

tdfz = tdfx.copy().groupby(by=['sequence','src','dst']).max().sort_values('epoch').reset_index()
dumpdf(tdfz)

KeyError: '192.168.25.13'

In [ ]:
''' Segmentation inspection '''
tdfx = seg_df.copy()
dumpdf(tdfx)

In [ ]:
''' EPC to Cloudlet '''
wdf = waterspout_icmp_df.copy()
cdf = cloudlet_icmp_df.copy()
seqminset = list(set(waterspout_icmp_df.sequence) \
    .intersection((set(cloudlet_icmp_df.sequence))))
# print(seqminset)
wdf['NAME'] = 'waterspout'
cdf['NAME'] = 'cloudlet'
jdf = pd.concat([wdf,cdf])
jdf = jdf[jdf.sequence.isin(seqminset)]
jdf = drp_lst(jdf,['data_time','identifier','htime'])
jdf['sepoch'] = jdf.epoch.map(lambda col: "{}".format(col))
# dumpdf(wdf)
# dumpdf(cdf)
display_all(jdf.sort_values('epoch')[['NAME','src','dst','sequence','sepoch']])